# 简介

[Pydantic](https://docs.pydantic.dev/latest/)用于模式验证,类似`npm`的`zod`库.


# 示例

## 基本使用

In [ ]:
import json
from datetime import datetime

import yaml
from pydantic import BaseModel, PositiveInt


class User(BaseModel):
    id: int
    name: str = 'John Doe'
    signup_ts: datetime | None
    tastes: dict[str, PositiveInt]


external_data = {
    'id': 123,
    'signup_ts': '2019-06-01 12:22',
    'tastes': {
        'wine': 9,
        b'cheese': 7,
        'cabbage': '1',
    },
}

user = User(**external_data)

print(user.id)
#> 123
print(user.model_dump())
"""
{
    'id': 123,
    'name': 'John Doe',
    'signup_ts': datetime.datetime(2019, 6, 1, 12, 22),
    'tastes': {'wine': 9, 'cheese': 7, 'cabbage': 1},
}
"""

## 验证失败

验证失败时会抛出`ValidationError`异常,可以通过`e.errors()`获取错误信息.

In [ ]:
# continuing the above example...

from pydantic import ValidationError
import json


class User(BaseModel):
    id: int
    name: str = 'John Doe'
    signup_ts: datetime | None
    tastes: dict[str, PositiveInt]


external_data = {'id': 'not an int', 'tastes': {}}

try:
    User(**external_data)
except ValidationError as e:
    print(json.dumps(e.errors(), indent=4))
    """
    [
        {
            'type': 'int_parsing',
            'loc': ('id',),
            'msg': 'Input should be a valid integer, unable to parse string as an integer',
            'input': 'not an int',
            'url': 'https://errors.pydantic.dev/2/v/int_parsing',
        },
        {
            'type': 'missing',
            'loc': ('signup_ts',),
            'msg': 'Field required',
            'input': {'id': 'not an int', 'tastes': {}},
            'url': 'https://errors.pydantic.dev/2/v/missing',
        },
    ]
    """

## 数据类模板

In [ ]:
from pydantic import BaseModel


class ClassName(BaseModel):
    int_no_default: int
    """
    整数,无默认值
    """
    int_with_default: int = 42
    """
    整数,有默认值
    """
    str_no_default: str
    """
    字符串,无默认值
    """
    str_with_default: str = 'foobar'
    """
    字符串,有默认值
    """

## 将`json`转换为`pydantic`类

In [6]:
from pydantic import BaseModel
import json

json_string = '''
{
    "int_no_default": "1",
    "str_no_default": "hello"
}
'''

# 将 JSON 字符串解析为字典
json_data = json.loads(json_string)

# 将字典转换为 Pydantic 类实例
class_instance = ClassName(**json_data)

print( class_instance)

int_no_default=1 int_with_default=42 str_no_default='hello' str_with_default='foobar'


## 将`yaml`转换为`pydantic`类

In [12]:
from pydantic import BaseModel
import yaml

class OpenAIConfig(BaseModel):
    base_url: str
    """OpenAI 服务的基础 URL"""

    token: str
    """OpenAI 服务的访问令牌"""

class KidEngStoryConfig(BaseModel):
    openai: OpenAIConfig
    """OpenAI 配置"""
def load_config(file_path):
    with open(file_path, 'r') as file:
        return yaml.safe_load(file)

config = load_config(r'D:\Workspace\aiworld\py_automation\kid_eng_story\app.yml')

class_instance = KidEngStoryConfig(**config)

print( class_instance)

openai=OpenAIConfig(base_url='https://openaiproxy3.nexcode.top/openai/chat/completions', token='sk-proj-k4mYRSUVs0N5oWgRA2WlT3BlbkFJdFjLhBybUmxwucB4N68W')


### 将`yaml`转换为泛型`pydantic`类

In [13]:
from typing import TypeVar, Type

# 定义泛型类型变量
T = TypeVar('T', bound=BaseModel)

def load_yaml_as_pydantic(file_path: str, model: Type[T]) -> T:
    """
    读取 YAML 文件并将其转换为 Pydantic 类实例.

    :param file_path: YAML 文件路径
    :param model: Pydantic 泛型类型
    :return: Pydantic 类实例
    """
    with open(file_path, 'r') as file:
        yaml_data = yaml.safe_load(file)
    
    # 将 YAML 数据转换为 Pydantic 类实例
    return model(**yaml_data)

config = load_yaml_as_pydantic(r'D:\Workspace\aiworld\py_automation\kid_eng_story\app.yml', KidEngStoryConfig)
print(config)

openai=OpenAIConfig(base_url='https://openaiproxy3.nexcode.top/openai/chat/completions', token='sk-proj-k4mYRSUVs0N5oWgRA2WlT3BlbkFJdFjLhBybUmxwucB4N68W')
